<a href="https://colab.research.google.com/github/kylecshan/xplore_idm/blob/master/getBuiltClusters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import packages and authenticate

In [0]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [0]:
# Earth Engine install to notebook VM.
!pip install earthengine-api

In [0]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/tQHoEGQ4hD9Q9tknqYHMjrzBVEQAf1oTgRlh40be827R7K1toCsWWZ8

Successfully saved authorization token.


## Configuration

In [0]:
# Specify names locations for outputs in Cloud Storage. 
BUCKET = 'xplore_idm_fall2019'
FOLDER = 'built'

## Get images
### Nigeria country border

In [0]:
nigeria = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
  .filterMetadata('country_co', 'equals', 'NI') \
  .first();
nigeria = ee.Feature(nigeria).geometry();

### Built environment

In [0]:
dataset = ee.Image('JRC/GHSL/P2016/BUILT_LDSMT_GLOBE_V1')
built = dataset.select('built').clip(nigeria)

### Upscale night lights to match landsat, and sample data

In [0]:
from google.colab import files
uploaded = files.upload()

Saving dhs_gps.csv to dhs_gps.csv


In [0]:
import io
import pandas as pd
clusters = pd.read_csv(io.BytesIO(uploaded['dhs_gps.csv']))
print(clusters)
vaccines = clusters.columns[1:12]
print(vaccines)
clusters = clusters.to_numpy()
n = clusters.shape[0]

     cluster_id  latitude  ...  health_card_mean  any_vacc_mean
0             1  6.902270  ...          0.280000       0.960000
1             2  7.854454  ...          0.571429       1.000000
2             3  7.781454  ...          0.500000       0.777778
3             4  7.176902  ...          0.142857       0.769231
4             5  7.148612  ...          0.407407       0.764706
..          ...       ...  ...               ...            ...
884         900  7.368560  ...          0.947368       0.916667
885         901  8.570109  ...          0.016129       0.370968
886         902  8.765458  ...          0.586207       0.562500
887         903  8.660406  ...          0.454545       0.400000
888         904  8.687992  ...          0.735294       0.590909

[889 rows x 25 columns]
Index(['latitude', 'longitude', 'bcg_sum', 'measles_sum', 'dpt1_sum',
       'dpt2_sum', 'dpt3_sum', 'polio0_sum', 'polio1_sum', 'polio2_sum',
       'polio3_sum'],
      dtype='object')


In [0]:
def clipExport(lat, lon, filename):
  bounds = ee.Geometry.Point(lon, lat).buffer(5000).bounds()
  task = ee.batch.Export.image.toCloudStorage(
    image = built,
    region = bounds.getInfo()['coordinates'],
    description = filename,
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + filename,
    scale = 38,
    fileFormat = 'GeoTIFF',
    fileDimensions = 1024
  )
  task.start()
  return

In [0]:
for k in range(200, 889):
  description = 'built_' + str(int(clusters[k, 0])).rjust(3,'0')
  clipExport(clusters[k, 1], clusters[k, 2], description)


In [0]:
clusters[k,3]

47.0

### Check task status

In [0]:
# Print all tasks.
import pprint
pp = pprint.PrettyPrinter()
pp.pprint(ee.batch.Task.list()[:150])  

[<Task EXPORT_IMAGE: built_202 (READY)>,
 <Task EXPORT_IMAGE: built_201 (READY)>,
 <Task EXPORT_IMAGE: built_200 (READY)>,
 <Task EXPORT_IMAGE: built_199 (READY)>,
 <Task EXPORT_IMAGE: built_198 (READY)>,
 <Task EXPORT_IMAGE: built_197 (READY)>,
 <Task EXPORT_IMAGE: built_196 (READY)>,
 <Task EXPORT_IMAGE: built_195 (READY)>,
 <Task EXPORT_IMAGE: built_194 (READY)>,
 <Task EXPORT_IMAGE: built_193 (READY)>,
 <Task EXPORT_IMAGE: built_192 (READY)>,
 <Task EXPORT_IMAGE: built_191 (READY)>,
 <Task EXPORT_IMAGE: built_190 (READY)>,
 <Task EXPORT_IMAGE: built_189 (READY)>,
 <Task EXPORT_IMAGE: built_188 (READY)>,
 <Task EXPORT_IMAGE: built_187 (READY)>,
 <Task EXPORT_IMAGE: built_186 (READY)>,
 <Task EXPORT_IMAGE: built_185 (READY)>,
 <Task EXPORT_IMAGE: built_184 (READY)>,
 <Task EXPORT_IMAGE: built_183 (READY)>,
 <Task EXPORT_IMAGE: built_182 (READY)>,
 <Task EXPORT_IMAGE: built_181 (READY)>,
 <Task EXPORT_IMAGE: built_180 (READY)>,
 <Task EXPORT_IMAGE: built_179 (READY)>,
 <Task EXPORT_IM

'008'